In [27]:
from nltk.tokenize import sent_tokenize
from string import punctuation
from collections import defaultdict, Counter
from razdel import tokenize as razdel_tokenize
from scipy.sparse import csr_matrix

import numpy as np

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

<h2>Задание 1</h2>

In [13]:
with open('2ch_corpus.txt','r',encoding='utf-8') as inp:
    dvach = inp.read()

with open('lenta.txt','r',encoding='utf-8') as inp:
    news = inp.read()

In [29]:
sentences_dvach = [['<pad>','<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_news = [['<pad>','<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

Инкапсулируем функционал 3-граммной модели в класс, а для хранения условных вероятностей воспользуемся разреженной матрицей:

In [30]:
class TriGramModel:
    def __init__(self, corpus,
                 start_bigram = ('<pad>','<start>'),
                 end_token = '<end>'):
        self.corpus = corpus
        self.start_bigram = start_bigram
        self.end_token = end_token
        self.create_matrix()
    

    def create_matrix(self):
        unigrams = Counter(token for sent in self.corpus for token in sent[2:])
        
        self.id2word = [word for word in unigrams]
        self.word2id = {word:i for i,word in enumerate(self.id2word)}
        
        bigrams = Counter((token1,token2) for sent in self.corpus for token1,
                               token2 in zip(sent[:-2],sent[1:-1]))

        self.id2bigram = [bigram for bigram in bigrams]
        self.bigram2id = {bigram:i for i,bigram in enumerate(self.id2bigram)}
        
        trigrams = Counter((token1,token2,token3) for sent in self.corpus for token1,
                            token2, token3 in zip(sent[:-2],sent[1:-1],sent[2:]))
        
        trigram_keys = [i for i in trigrams]
        row_ind = [self.bigram2id[(i[0],i[1])] for i in trigram_keys]
        col_ind = [self.word2id[i[2]] for i in trigram_keys]
        data = [trigrams[i]/bigrams[(i[0],i[1])] for i in trigram_keys]
        
        self.matrix = csr_matrix((data,(row_ind,col_ind)), shape=(len(bigrams),len(unigrams)))
        
    
    def generate_text(self, length=100):
        text = []
        bigram = self.start_bigram

        for i in range(length):
            bigram_id = self.bigram2id[bigram]
            chosen = np.random.choice(self.matrix.shape[1], p=self.matrix[bigram_id].toarray()[0])
            
            new_token = self.id2word[chosen]
            text.append(new_token)

            if new_token == self.end_token:
                bigram = self.start_bigram
            else:
                bigram = (bigram[1], new_token)

        return ' '.join(text)

In [37]:
model_dvach = TriGramModel(sentences_dvach)

In [38]:
model_dvach.matrix.shape

(978238, 189516)

Посмотрим на выдачу на основе "двачевского" текста:

In [45]:
print(model_dvach.generate_text().replace('<end>','\n\n'))

факты 

 у вас синусы-косинусы зря столько раз дох что уже давно могилизовался бы за то что я самовлюбленный болван 

 если красной ручкой ваше дело передано в прокуратуру а то и другое 

 каждая вторая бабка от 30 августа потом снова будет ручкаться с эрдоганом 

 собираюсь идти дальше 

 надеюсь тебя поймают 

 это уже аксиома 

 в обоих был рак то еще и директор там прикольный дядька не высокомерный обычно-кун такой себе 

 сижу тут чтобы доказать вам вашу ничтожность и превосходство тян над кунами 

 хорошь вестись на толстоту тема про 4 20 pm вроде кошаков


И на основе новостных текстов:

In [41]:
model_news = TriGramModel(sentences_news)

In [42]:
model_news.matrix.shape

(748147, 116303)

In [47]:
print(model_news.generate_text().replace('<end>','\n\n'))

в то же время ар напоминает что клинтоновская администрация обещала обеспечить российских ядерщиков подобной техникой если москва не изменит своей позиции которая гораздо мягче в отношении россии 

 кроме того утверждены президиум фракции из 11 человек будут иметь совокупно 65-процентный пакет акций компании notharvard com уже перегруженная именами и разборками вокруг этих запасов может причинить серьезный ущерб городу-курорту сочи 

 помимо этого райков сообщил что начальник флотской контрразведки оклеветал его в 2006 году и составляющего не более чем зыбкой на сайте fnsi 

 в заключение президент франции жак ширак сообщает интерфакс россиянин признан виновным в организации терактов в стране отнюдь


<b>Вывод</b>: По сравнению с биграммной моделью, реализованной в <a href="">семинарской тетрадке</a>, предложения, сгенерированные триграммной моделью получаются борлее длинными, согласованными (допустимыми грамматикой русской языка) и осмысленными

<h2>Задание 2</h2>

In [11]:
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from pymorphy2 import MorphAnalyzer
from collections import Counter, defaultdict
import re
from string import punctuation
from nltk.corpus import stopwords

stops = set(stopwords.words('russian') + ["это", "весь"])
morph = MorphAnalyzer()

def normalize(text):
    tokens = re.findall('[а-яёa-z0-9]+', text.lower())
    normalized_text = [morph.parse(word)[0].normal_form for word \
                                                            in tokens]
    normalized_text = [word for word in normalized_text if len(word) > 2 and word not in stops]
    
    return normalized_text

def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

In [7]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser, original_scorer, npmi_scorer
from math import log

Вспомним формулу PMI и имплементируем эту метрику:

$PMI=log\frac{p(xy)}{p(x)p(y)}=log\frac{count(xy)/N}{(count(x)/N)(count(y)/N)} = log(count(xy))+log(N)-log(count(x))-log(count(y))$

In [8]:
def PMI(worda_count, wordb_count, bigram_count,
       len_vocab, min_count, corpus_word_count):
    return log(bigram_count)+log(corpus_word_count)-log(worda_count)-log(wordb_count)

In [14]:
corpus = preprocess(news)

Выберем предложение на котором будем тестировать модели Phrases:

In [53]:
test_sentence = corpus[10]

In [54]:
print(test_sentence)

['русский', 'инвалид', 'сентябрь', '1914', 'год', 'министерство', 'народный', 'просвещение', 'вид', 'происходить', 'чрезвычайный', 'событие', 'признать', 'соответственный', 'день', 'годовщина', 'день', 'рождение', 'лермонтов', 'октябрь', '1914', 'год', 'ограничиться', 'совершение', 'учебный', 'заведение', 'панихида', 'поэт', 'отложить', 'празднование', 'юбилей', 'благоприятный', 'время']


Ожидаем получить следующие значимые N-граммы - <i>министерство народного просвещения, 1914 год, чрезвычайное событие, день рождения, учебное заведение, празднование юбилея, благоприятное время</i>

In [23]:
model = Phrases(corpus,threshold=2,scoring=PMI)

In [56]:
for threshold in range(10):
    model1 = Phrases(corpus,threshold=threshold,scoring=PMI)
    phraser1 = Phraser(model1)
    model2 = Phrases(phraser1[corpus],threshold=threshold,scoring=PMI)
    phraser2 = Phraser(model2)
    print(f"Threshold={threshold}: {model2[test_sentence]}")

Threshold=0: ['русский_инвалид', 'сентябрь', '1914', 'год_министерство', 'народный', 'просвещение', 'вид_происходить', 'чрезвычайный', 'событие', 'признать', 'соответственный', 'день_годовщина', 'день_рождение', 'лермонтов', 'октябрь', '1914', 'год', 'ограничиться', 'совершение', 'учебный_заведение', 'панихида', 'поэт', 'отложить', 'празднование', 'юбилей', 'благоприятный', 'время']
Threshold=1: ['русский_инвалид', 'сентябрь', '1914', 'год', 'министерство', 'народный', 'просвещение', 'вид', 'происходить', 'чрезвычайный', 'событие_признать', 'соответственный', 'день_годовщина', 'день_рождение', 'лермонтов', 'октябрь', '1914', 'год_ограничиться', 'совершение_учебный', 'заведение_панихида', 'поэт', 'отложить', 'празднование', 'юбилей', 'благоприятный', 'время']
Threshold=2: ['русский_инвалид', 'сентябрь', '1914', 'год', 'министерство', 'народный', 'просвещение', 'вид_происходить', 'чрезвычайный_событие', 'признать', 'соответственный', 'день_годовщина', 'день_рождение', 'лермонтов', 'октяб

<b>Вывод</b>: Коллокация <i>министерство народного просвещения</i> не находится моделью ввиду малой представленности исторических новостей в корпусе. При повышении порога PMI до 1 пропадает ложная коллокация <i>год просвещения</i>, а при повышении до 2 коллокация <i>событие признать</i>. При повышения порога до 9 модель перестаёт обнаруживать коллокации в тестовом предложении. Из ожидаемых коллокаций были обнаружены <i>чрезвычайное событие</i>(при пороге=3), <i>день рождения</i> (при пороге от 1 до 5), <i>учебное завдение</i> (при пороге равном 0 и от 2 до 8), <i>празднование юбилея</i> (при пороге от 6 до 8). При значениях порога от 0 до 9 включительно модель не обнаружила ни одной значимой 3-граммы. Также следует отметить, что при повышении порога некоторые словосочетания, не прошедшие порог коллокационной силы на первом уровне проходили его на втором (в уже забиграммленном тексте), в связи с чем повышение порога могло приводить к появлению новых коллокаций (в отличие от одноуровневой модели, где повышение порога только сокращало было ранее обнаруженный набор коллокаций).